In [ ]:
!pip install llama-cpp-python

In [11]:
from llama_cpp import Llama
from tqdm import tqdm

import re
import spacy

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
MODEL_PATH = "../../../llama.cpp/models/vicuna-7b-1.1.ggmlv3.q4_0.bin"
llm = Llama(model_path=MODEL_PATH, verbose=False)

llama.cpp: loading model from ../../../llama.cpp/models/vicuna-7b-1.1.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 3917.73 MB (+  256.00 MB per state)
llama_new_context_with_model: kv self size  =  

In [14]:
def style_transfer(style, text, llm, nlp, max_tokens, 
                   temperature = 0.8, top_p=0.95,
                   top_k=40, reset=True):
    doc = nlp(text)
    sents = [s.text for s in doc.sents]
    start = 0
    end = len(sents)
    step = 6
    
    output_list = []
    for i in tqdm(range(start, end, step)):
        prompt_sentences = " ".join(sents[i:i+step]).replace('\n', '')
        prompt = f"""Q: Can you re-write the following in the style of {style}: 
        
        '{prompt_sentences}'

        A:
        """
        output = llm(prompt, echo=False, 
                     max_tokens = max_tokens,
                     temperature = temperature, 
                     top_p=top_p, top_k=top_k)
        output_list.append((prompt, output))
        if reset:
            llm.reset()
    return output_list

In [15]:
text = """
Different groups of people cycle through the farms in neighborhoods outside downtown, ready to provide food for thousands of people occupying Woodruff Park. 
A warehouse on the west side has trucks and teams to drive to abandoned hotels and industrial waste facilities, gathering “raw” material — metal, lumber, kitchen equipment — that can be used to build brick ovens and fix up the new building. 
A partisan cafe downtown functions as an entry point for visitors and newcomers, as well as a drop-in point for insurgents from around the state, the region, the country, and even the world. The dance club lets people in to blend with the crowd after a rowdy demo while giving them a way to blow off some steam. 
Pirate radio transmitters broadcast from secret locations outside of the city to spread sedition and heresy into the heart of a great metropolis. University copy machines are hacked for free prints for this weekend’s assembly — the print shop is already running overtime. 
A friend walks out of the store with a backpack full of goods and a knowing wink. Doctors and herbalists are at hand, equipped to deal with any injuries that might ensue from tonight’s riot, well trained from treating common ailments and injuries. 
The family lake house is repurposed to sleep a hundred for a summer strategy meeting. Slowly, something is growing.
"""

In [38]:
outputs = style_transfer("Jean Baudrillard", text, llm, nlp, max_tokens=512)

100%|█████████████████████████████████████████████████████████| 4/4 [03:48<00:00, 57.01s/it]


In [39]:
def extract_choices(outputs):
    choices = []
    for o in outputs:
        choices.append(o[1]['choices'][0]['text'].replace("\n", ""))
    return " ".join(choices).replace("\n", "")

In [40]:
extract_choices(outputs)

"```The Bayou Oaks golf course, a symbol of civilization's relentless expansion into nature, was attacked by an anonymous group of rebels in the darkest hours of July 2nd, under the watchful gaze of the full moon. The concrete was poured into the holes, the greens were dug into, and messages of rebellion were left behind as a testament to the solidarity of the saboteurs with the forest defenders in Weelaunee. The act was a direct challenge to the authority of the cops who rule over City Park and everywhere else they have made their presence felt. Power to the forest defenders! Stop Cop City!```         The redevelopment of the Bayou Oaks golf course in the aftermath of Katrina was met with widespread opposition, chief among them the destruction of a fallow park area that served as a last refuge for native animals already struggling under the weight of citywide development. A decade of rewilding since the storm has offered a glimpse into a world that continues to exist in spite of 300 y